NUMFLOORS / Footprints floors query

In [ ]:
WITH pluto AS (
	SELECT CAST(bbl AS TEXT),
		bldgclass,
		CAST(numfloors AS NUMERIC) AS numfloors,
		CASE
    		WHEN SUBSTRING(bldgclass,1,1) IN ('A', 'B', 'C', 'D') THEN 10
			ELSE 12
		END AS floor_height,
		numbldgs,
		address,
		geom
	FROM dcp.pluto201
	WHERE CAST(numbldgs AS NUMERIC) < 2
	AND CAST(numfloors AS NUMERIC) > 0
), footprints AS (
    SELECT mpluto_bbl,
		MAX(CAST(heightroof AS NUMERIC)) AS heightroof
	FROM dcp.footprints 
	WHERE CAST(heightroof AS NUMERIC) > 0
	GROUP BY mpluto_bbl
), housing AS (
	SELECT bbl,
		job_number,
		job_type,
		job_status,
		date_complete
	FROM dcp.housing
	WHERE job_type = 'New Building' AND SUBSTRING(date_complete,1,4) > '2013'
), calc_differential AS (
	SELECT p.bbl,
		p.bldgclass,
		p.address,
		p.numbldgs,
		p.numfloors,
		ROUND(f.heightroof / p.floor_height, 3) AS footprints_floors,
		f.heightroof,
		p.floor_height,
		ABS(p.numfloors - ROUND(f.heightroof / p.floor_height, 3)) AS differential,
		p.geom
	FROM pluto p
	INNER JOIN footprints f
	ON p.bbl = f.mpluto_bbl
	WHERE p.bbl NOT IN (SELECT h.bbl FROM housing h)
)
SELECT * FROM calc_differential
WHERE (differential > (2 * numfloors) OR differential > (2 * footprints_floors)) AND (numfloors > 10 OR footprints_floors > 10)
AND SUBSTRING(bldgclass,1,1) NOT IN ('M', 'Q')
ORDER BY numfloors DESC;

PLUTO records having no match on footprints, and no recent 'New Building' permit.

In [ ]:
SELECT p2.bbl, p2.bldgclass, p2.address, p2.numbldgs, p2.numfloors, 
null, null, null, null, p2.geom
FROM dcp.pluto201 p2
WHERE NOT EXISTS 
	(SELECT f2.mpluto_bbl 
 		FROM dcp.footprints f2
		WHERE f2.mpluto_bbl = CAST(p2.bbl AS TEXT))
AND p2.numbldgs < 2
AND p2.numfloors > 50
AND CAST(p2.bbl AS TEXT) NOT IN (SELECT h.bbl FROM dcp.housing h WHERE job_type = 'New Building' AND SUBSTRING(date_complete,1,4) > '2013')
AND SUBSTRING(bldgclass,1,1) NOT IN ('M', 'Q')

ORDER BY numfloors DESC;